# Introduction to Google's TensorFlow

This Jupyter notebook contains a set of examples as part of a presentation to help developers get a feel for using the TensorFlow client interface.

## Hello World!

In good fashion we will start our hands lab by creating a TensorFlow graph with a single node that produces the constant _Hello, TensorFlow!_.

In [ ]:
import tensorflow as tf

with tf.Session() as session:
    print session.run(tf.constant('Hello, TensorFlow!'))

## Getting Started with Variables and Operations

In TensorFlow Variables are used to maintain state across executions of the graph. Variables are not initialized when they are defined so we must be careful to tell TensorFlow to initialize all the Variables before we attempt to use them. Below is an example countdown latch.

In [ ]:
# Define our latch.
latch = tf.Variable(10, name = 'latch')

# Define a constant by which we decrement our latch.
one = tf.constant(1)

# Apply a subtraction operation.
new_value = tf.sub(latch, one)

# Update the state of our latch.
update = tf.assign(latch, new_value)

# IMPORTANT to run before using Variables.
init_op = tf.initialize_all_variables()

# Execute our graph.
with tf.Session() as session:
    session.run(init_op)
    for _ in range(10):
        session.run(update)
        print session.run(latch)
        
    

## Feeding Tensors into a Graph

So far we have provided tensors into the graph by either using constants or variables but TensorFlow provides a feed mechanism for patching tensors directly into any operation.

In [ ]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
output = tf.mul(a, b)

with tf.Session() as session:
  print(session.run([output], feed_dict={a: [5.], b: [5.]}))

## Fetching Tensors from a Graph

In the previous example we fetched a single tensor but it is also possible to fetch multiple tensors by specifying them in the run method.

In [ ]:
a = tf.constant(1)
b = tf.constant(2)

add = tf.add(a, b)
mul = tf.mul(a, b)

with tf.Session() as session:
  print(session.run([add, mul]))

## Choosing a Device

TensorFlow allows us to distribute our graph across many devices seemlessly but there are some cases where we need to help. If your device has a GPU installed TensorFlow will automatically use your GPU for as many  operations as possible. If your device has more than one GPU installed you have to explicitly tell TensorFlow how to distribute operations across the GPUs.

Since we don't have any guarantees that TensorFlow will be running on a device with a GPU installed the example below will use CPU cores instead to demonstrate how to chose a device on which to execute operations.

In [ ]:
# with tf.device("/gpu:0"): This would use the first GPU on the sysytem.
# with tf.device("/gpu:1"): This would use the second GPU on the system.
with tf.device('/cpu:0'):
    M = tf.constant([[2., 3.]])
    N = tf.constant([[4.],[5.]])
    product = tf.matmul(M, N)
    
with tf.Session() as session:
    print(session.run(product))

## Machine Learning with TensorFlow

In order to bring it all together below is an implementation of the nearest neighbor algorithm using TensorFlow

In [ ]:
import input_data
import numpy as np

# Import mnist data.
mnist = input_data.read_data_sets('/tmp/mnist/', one_hot = True)
train_x, train_y = mnist.train.next_batch(10000)
test_x, test_y = mnist.test.next_batch(100)

# Squash the input images into vectors.
train_x = np.reshape(train_x, newshape = (-1, 28 * 28))
test_x = np.reshape(test_x, newshape = (-1, 28 * 28))

# Build the graph.
neighbors = tf.placeholder('float', [None, 784])
x = tf.placeholder('float', [784])
distance = tf.reduce_sum(tf.abs(tf.add(neighbors, tf.neg(x))), reduction_indices = 1) # Calculate Manhattan Distance
prediction = tf.arg_min(distance, 0) # Get the Nearest neighbor

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    
    # loop over test data
    accuracy = 0.
    for i in range(len(test_x)):
        # Get nearest neighbor
        nn = session.run(prediction, feed_dict={ neighbors: train_x, x: test_x[i,:] })
        # Get nearest neighbor class label and compare it to the target label
        print 'Prediction:', np.argmax(train_y[nn]), 'Target:', np.argmax(test_y[i])
        # Calculate accuracy
        if np.argmax(train_y[nn]) == np.argmax(test_y[i]):
            accuracy += 1. / len(test_x)
    print 'Accuracy:', accuracy